In [3]:
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.1.0
Keras Version: 2.2.4-tf

Python 3.7.9 (default, Aug 31 2020, 17:10:11) [MSC v.1916 64 bit (AMD64)]
Pandas 1.1.1
Scikit-Learn 0.23.2
GPU is NOT AVAILABLE


In [4]:
"""
my format of json

{
"image1.jpg":{
        "filename":"image1.jpg",
        "size":123456,
        "regions":[
            {
                "shape_attributes":{
                    "all_points_x":[
                        675,
                        808,
                        957,
                        967,
                        929,
                        791,
                        678,
                        703
                    ],
                    "all_points_y":[
                        543,
                        518,
                        492,
                        722,
                        760,
                        760,
                        760,
                        647
                    ],
                    "name":"polygon"
                },
                "region_attributes":{
                    "Object":"class1",
                }
            },...all other regions..{}]
    }, ..all other images..., {}
}

"""

import glob, os, cv2
import numpy as np
import multiprocessing as mp
from PIL import Image
from scipy.spatial import Delaunay as Delaunay2D
import json


print("ok")

ok


In [ ]:


def calculateDelauneyPoints(points):
    points= np.array(points).astype(int)
    points= [list(item) for item in points]
    dt = Delaunay2D()
    for s in points:
        dt.addPoint(s)
    coord, tris= dt.exportDT()
    return np.array(coord)

def maskImage(filename, polygons, o_list):
    root_path= '/3dircab/'
    image = cv.imread(root_path+"train/"+filename)
    mask = np.zeros(shape = image.shape, dtype = "uint8")
    i= 0
    for points in polygons:
        points= [(int(x), int(y)) for x,y in points]
        try:
            rect= calculateDelauneyPoints(points)
        except:
            continue
        if o_list[i] in ['class1']:
            cv.drawContours(mask,[rect], -1,(1, 1, 1),cv.FILLED)
        elif o_list[i] in ['class2']:
            cv.drawContours(mask,[rect], -1,(2, 2, 2),cv.FILLED)
        elif o_list[i] in ['class3']:
            cv.drawContours(mask,[rect], -1,(3, 3, 3),cv.FILLED)
        i+= 1
    
    cv.imwrite(root_path+"dataset/images/"+os.path.splitext(filename)[0]+".png", image)
    cv.imwrite(root_path+"dataset/masks/"+os.path.splitext(filename)[0]+".png", mask)
    
    print(filename+"ok")   
def getRegionProperties(region):
    shape_attributes= region["shape_attributes"]
    region_attributes= region["region_attributes"]
    objects= region_attributes["Object"]
    regions= ['msMask']
    all_points_x= shape_attributes["all_points_x"]
    all_points_y= shape_attributes["all_points_y"]
    coordinates= []
    for i in range(0, len(all_points_x)):
        coordinates.append((all_points_x[i], all_points_y[i]))
    
    return (objects, coordinates)

def parallelizePlotting(data, region_mappings, json_data):
    
    polygon_coordinates= {}
    polygons= []
    img_json_data= json_data[data]
    
    filename= img_json_data["filename"]
    
    # Open the original image here
    image_matrix= np.array(Image.open('../data/all_images/'+filename), dtype= np.uint8)
    region_data= img_json_data["regions"]
    objects_list= []
    
    for region in region_data:
        objects, coordinates= getRegionProperties(region)
        if coordinates is not None:
            polygons.append(coordinates)
            polygon_coordinates[objects]= coordinates
            objects_list.append(objects)
    
    # Masking the images
    maskImage(filename, polygons, objects_list)
    return (filename, polygon_coordinates)



def main():
    
    root_path= "C:/workplace/data/"

    json_file= open(root_path+'via_region_data.json')
    json_data= json.load(json_file)
    
    # for multiprocessing
    pool = mp.Pool(mp.cpu_count()-1)
    
    def resultCallback(item):
        return
        
    for data in json_data:
        pool.apply_async(parallelizePlotting, args=(data, json_data), callback= resultCallback)
    
    pool.close()
    pool.join()



if __name__ == "__main__":
    main()

In [1]:

from skimage import draw
from skimage import io
import numpy as np
import urllib.request
import json
import logging
import os
import sys


###################  INSTALLATION NOTE #######################
##############################################################

## pip install scikit-image
## pip install numpy

###############################################################
###############################################################

#enable info logging.
logging.getLogger().setLevel(logging.INFO)

def poly2mask(blobs, c, path_to_masks_folder, h, w, label, idx):
    mask = np.zeros((h, w))
    for l in blobs:
        fill_row_coords, fill_col_coords = draw.polygon(l[1], l[0], l[2])
        mask[fill_row_coords, fill_col_coords] = 1
    io.imsave(path_to_masks_folder + "/" + str(c) + "_" + label + "_" + str(idx) + ".png", mask)


def convert_dataturks_to_masks(path_to_dataturks_annotation_json, path_to_original_images_folder, path_to_masks_folder):
    # make sure everything is setup.
    if (not os.path.isdir(path_to_original_images_folder)):
        logging.exception(
            "Please specify a valid directory path to download images, " + path_to_original_images_folder + " doesn't exist")
        return
    if (not os.path.isdir(path_to_masks_folder)):
        logging.exception(
            "Please specify a valid directory path to write mask files, " + path_to_masks_folder + " doesn't exist")
        return
    if (not os.path.exists(path_to_dataturks_annotation_json)):
        logging.exception(
            "Please specify a valid path to dataturks JSON output file, " + path_to_dataturks_annotation_json + " doesn't exist")
        return
    
    print(path_to_dataturks_annotation_json) 
    f = open(path_to_dataturks_annotation_json)
    train_data = f.readlines()
    train = []
    data = json.load(path_to_dataturks_annotation_json)
    c = 0
    for objects in train:
        blobs = []
        classes = {}
        image = objects['content'][objects['content'].rfind('_') + 1:objects['content'].rfind('.')]
        # download the images from given url
        urllib.request.urlretrieve(objects['content'], path_to_original_images_folder + "/image" + str(c) + ".jpg")
        annotations = objects['annotation']

        for annot in annotations:
            blobs = []
            label = annot['label']
            if (label != ''):
                if label not in classes:
                    classes[label] = 0

                points = annot['points']
                h = annot['imageHeight']
                w = annot['imageWidth']
                x_coord = []
                y_coord = []
                l = []
                for p in points:
                    x_coord.append(p[0] * w)
                    y_coord.append(p[1] * h)
                shape = (h, w)
                l.append(x_coord)
                l.append(y_coord)
                l.append(shape)
                blobs.append(l)
                poly2mask(blobs, c, path_to_masks_folder, annot['imageHeight'], annot['imageWidth'], label,
                          classes[label])
                classes[label] += 1
        c += 1


if (len(sys.argv) < 4):
    print(
    "Please provide path to dataturks json file, path to store ground truth images and path to store mask images in this order.")
    exit(0)
#convert_dataturks_to_masks(sys.argv[1], sys.argv[2], sys.argv[3])

jsonF="C:\\workplace\\data\\3dircab\\train\\via_region_data.json"
originalPath="C:\\workplace\\data\\3dircab\\train\\"
maskPath="C:\\workplace\\data\\3dircab\\trainMask\\"
convert_dataturks_to_masks(jsonF, originalPath, maskPath)

Please provide path to dataturks json file, path to store ground truth images and path to store mask images in this order.
C:\workplace\data\3dircab\train\via_region_data.json


AttributeError: 'str' object has no attribute 'read'

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.vision import *
from fastai.callbacks.hooks import *
import PIL.Image as PilImage

def getClassValues(label_names):

    containedValues = set([])

    for i in range(len(label_names)):
        tmp = open_mask(label_names[i])
        tmp = tmp.data.numpy().flatten()
        tmp = set(tmp)
        containedValues = containedValues.union(tmp)
    
    return list(containedValues)

def replaceMaskValuesFromZeroToN(mask,containedValues):

    numberOfClasses = len(containedValues)
    newMask = np.zeros(mask.shape)

    for i in range(numberOfClasses):
        newMask[mask == containedValues[i]] = i
    
    return newMask

def convertMaskToPilAndSave(mask,saveTo):

    imageSize = mask.squeeze().shape

    im = PilImage.new('L',(imageSize[1],imageSize[0]))
    im.putdata(mask.astype('uint8').ravel())
    im.save(saveTo)

def convertMasksToGrayscaleZeroToN(pathToLabels,saveToPath):

    label_names = get_image_files(pathToLabels)
    containedValues = getClassValues(label_names)

    for currentFile in label_names:
        currentMask = open_mask(currentFile).data.numpy()
        convertedMask = replaceMaskValuesFromZeroToN(currentMask, containedValues)
        convertMaskToPilAndSave(convertedMask, saveToPath/f'{currentFile.name}')
    
    print('Conversion finished!')

ModuleNotFoundError: No module named 'fastai'